In [1]:
# core libraries
import pickle
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")
from datetime import datetime


from queries import final_features
from score_funcs import Score
sc = Score()


/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


In [2]:
os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


#### Data pull for scoring 

In [3]:
querySno = SQLQuery('snowflake')
df = querySno(final_features)

In [4]:
df.head()

,business_id,due_date,emi_no,median_running_balance_2nd_m,median_amount_debited_2m,distinct_debit_txns_1m,ratio_debit_credit_2m,ratio_ach_credit_freq_total_credit_1m,ratio_amt_drawn_mom,ratio_ach_debit_credit_freq_1m,ratio_amt_cashin_mom,ratio_revenue_1m_2m,distinct_cashins_1m_prev
0,e48c2153-6bec-4711-92b6-04f1fcbf8b62,2022-12-23,1,114.89,2.000,42,1.305130,0.736842,0.000000,0.000000,0.000000,0.177905,0.0
1,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-24,1,57.47,0.000,14,0.621232,0.062500,0.089158,0.000000,1.979837,0.043739,7.5
2,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-29,2,77.47,0.000,9,0.621232,0.043478,0.089158,0.000000,1.979837,0.043739,7.5
3,206dce8f-1527-48ca-b9aa-11fe81ed6c9c,2022-12-31,1,672.72,29.570,63,0.994326,0.357143,0.427960,0.166667,184.310000,1.824128,1.0
4,a848a226-51a3-4afb-a156-26513964b8e3,2022-12-31,1,1461.55,25.215,17,1.074374,0.333333,0.000000,0.000000,423.330000,6.590467,1.0


In [5]:
df.shape

(5725, 13)

In [6]:
data_params = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/data_params_v2.pkl')
model = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/redec_3.pkl')
transformer = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/scale_train_object.pkl')

In [ ]:
# df.isnull().sum()

In [8]:
sc.features

['distinct_debit_txns_1m',
 'median_amount_debited_2m',
 'ratio_debit_credit_2m',
 'ratio_ach_credit_freq_total_credit_1m',
 'ratio_ach_debit_credit_freq_1m',
 'median_running_balance_2nd_m',
 'ratio_revenue_1m_2m',
 'distinct_cashins_1m_prev',
 'ratio_amt_drawn_mom',
 'ratio_amt_cashin_mom']

In [11]:
df['emi_no'].value_counts()

2    1713
3    1167
4     363
Name: emi_no, dtype: int64

In [10]:
df = df[df['emi_no'].isin([2,3,4])]

In [12]:
df0 = sc.preprocess(df,sc.features,data_params,transformer)

In [13]:
df0 = sc.model_scoring_redec(df0,sc.features,model)

In [14]:
df0.head()

,business_id,due_date,emi_no,median_running_balance_2nd_m,median_amount_debited_2m,distinct_debit_txns_1m,ratio_debit_credit_2m,ratio_ach_credit_freq_total_credit_1m,ratio_amt_drawn_mom,ratio_ach_debit_credit_freq_1m,ratio_amt_cashin_mom,ratio_revenue_1m_2m,distinct_cashins_1m_prev,redec_proba,redec_score,redec_bin
0,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-29,2,-0.559013,-0.554756,-0.537224,-1.195758,-1.726065,-0.275416,-0.732551,-0.347744,-0.156665,2.881640,0.578490,421.509705,2
1,e48c2153-6bec-4711-92b6-04f1fcbf8b62,2023-01-23,2,-0.517349,-0.505696,-0.327072,0.376055,1.310021,-0.275490,-0.732551,-0.348842,-0.143452,-0.529549,0.527770,472.229921,2
2,4a95d0ac-791c-4c26-ba04-27529cf78788,2023-01-24,3,-0.156862,-0.554756,-0.545007,-0.891120,-1.884814,-0.275490,-0.732551,-0.348754,-0.082612,-0.004751,0.521497,478.502741,2
3,4a95d0ac-791c-4c26-ba04-27529cf78788,2023-01-29,4,-0.154727,-0.554756,-0.552790,-1.304407,-1.884814,-0.275490,-0.732551,-0.348754,-0.082612,-0.004751,0.491255,508.744785,2
4,16112c0e-9725-46b7-8f73-7c33242b93b6,2023-01-31,2,-0.575033,0.107546,-0.552790,0.000725,-1.884814,-0.275490,0.397582,-0.348842,0.046020,-0.529549,0.484909,515.091488,2


In [ ]:
# df0.to_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/new_dataset_scored.pkl')

In [ ]:
# from sqlalchemy.types import NVARCHAR
# # from conf.config import SQLQuery
# q = SQLQuery('snowflake')

# df0.to_sql(name='redec_scored_dataset',
#                  con=q.engine, 
#                  schema='prod_db.adhoc',
#                  if_exists='append', 
#                  index=False, 
#                  chunksize=16000, 
#                  method='multi',
#                  dtype={col_name: NVARCHAR for col_name in df0})

In [15]:
df0['redec_bin'].value_counts()

2    873
3    793
5    649
4    522
1    406
Name: redec_bin, dtype: int64

In [16]:
len(df0['business_id'].unique())

1713

In [17]:
df0.shape

(3243, 16)